In [1]:
# Imports
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import torch
import torch.nn as nn
from pytorch_forecasting.models import TemporalFusionTransformer
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
import torch
import torch.optim as optim
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
import lightning.pytorch as pl # Instead of import pytorch_lightning as pl

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
from pytorch_forecasting import DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

# Assuming 'train_data' and 'test_data' are already defined and preprocessed as per your previous code

#




/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet
from torch.utils.data import DataLoader
from pytorch_forecasting.data.encoders import NaNLabelEncoder


# Sample data provided
data_dict = {
    "Date": pd.to_datetime(["2004-08-19", "2004-08-20", "2004-08-23", "2004-08-24", "2004-08-25"]),
    "Open": [50.050049, 50.555557, 55.430431, 55.675674, 52.532532],
    "High": [52.082081, 54.594597, 56.796799, 55.855858, 54.054054],
    "Low": [48.028027, 50.300301, 54.579578, 51.836838, 51.991993],
    "Close": [50.220219, 54.209209, 54.754753, 52.487488, 53.053055],
    "Adj Close": [50.220219, 54.209209, 54.754753, 52.487488, 53.053055],
    "Volume": [44659096, 22834343, 18256126, 15247337, 9188602]
}

macro_data_dict = {
    "date": pd.to_datetime(["2004-08-19", "2004-08-20", "2004-08-23", "2004-08-24", "2004-08-25"]),
    "CPI": [85.6, 86.4, 87.2, 88.0, 88.6],
    "Mortgage_rate": [14.2050, 14.7900, 14.9040, 15.1325, 15.4000],
    "Unemp_rate": [7.5, 7.2, 7.5, 7.4, 7.4],
    "NASDAQ": [200.685556, 198.398636, 198.817619, 194.852105, 203.593182],
    "disposable_income": [4976.5, 4999.8, 4980.4, 4965.0, 4979.0],
    "Personal_consumption_expenditure": [1826.8, 1851.7, 1870.0, 1884.2, 1902.9],
    "personal_savings": [11.6, 11.4, 10.9, 10.8, 10.8]
}

# Convert dictionaries to DataFrames
data = pd.DataFrame(data_dict)
macro_data = pd.DataFrame(macro_data_dict)

# Merge the dataframes on the date
merged_data = pd.merge(data, macro_data, left_on="Date", right_on="date")

# Drop the extra date column
merged_data.drop(columns=["date"], inplace=True)

# Convert the "Date" column to a period index (daily frequency)
merged_data['time_idx'] = merged_data['Date'].dt.to_period("D").dt.to_timestamp().view(int) / 10**9
merged_data['time_idx'] = merged_data['time_idx'] - merged_data['time_idx'].min()
merged_data['time_idx'] = merged_data['time_idx'].astype(int)

# Create a simple categorical index for group_ids if not using "Open"
merged_data['group_id'] = merged_data['Date'].dt.strftime('%Y%m%d').astype(int)

# Create a simple categorical index for group_ids
merged_data['group_id'] = pd.Categorical(merged_data.index).codes

# Create the TimeSeriesDataSet
dataset = TimeSeriesDataSet(
    merged_data,
    time_idx="time_idx",
    target="Close",
    group_ids=["group_id"],
    min_encoder_length=1,  # minimum length of history
    max_encoder_length=3,  # maximum length of history
    min_prediction_length=1,
    max_prediction_length=1,
    static_categoricals=[],
    time_varying_known_reals=["CPI", "Mortgage_rate", "Unemp_rate", "NASDAQ", "disposable_income", "Personal_consumption_expenditure", "personal_savings"],
    time_varying_unknown_reals=["Open", "High", "Low", "Adj Close", "Volume"],
    target_normalizer=GroupNormalizer(
        groups=["group_id"], transformation="softplus"  # normalize by group
    )
)

# Prepare DataLoader
dataloader = DataLoader(dataset, batch_size=2)

/var/folders/d9/x_bnq71n58lbk4_znq_twz6h0000gn/T/ipykernel_12274/4176988995.py:40: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  merged_data['time_idx'] = merged_data['Date'].dt.to_period("D").dt.to_timestamp().view(int) / 10**9
/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/pytorch_forecasting/data/timeseries.py:1281: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 5 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__group_id': 0}, {'__group_id__group_id': 1}, {'__group_id__group_id': 2}, {'__group_id__group_id': 3}, {'__group_id__group_id': 4}]
  warnings.warn(


AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags

In [ ]:
from pytorch_forecasting import DeepAR

# Create the DeepAR model
deepar_model = DeepAR.from_dataset(
    dataset,
    learning_rate=1e-3,
    hidden_size=64,
    rnn_layers=2,
    dropout=0.1,
    loss='NormalDistributionLoss'
)

# Train the model
trainer = Trainer(max_epochs=50, gpus=0)
trainer.fit(deepar_model, train_dataloader=dataloader)

# Save the model
torch.save(deepar_model.state_dict(), './models/deepar_model.pth')


/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


AssertionError: Encoder and decoder variables have to be the same apart from target variable